# NB03 - Survival Analysis (Prepay AUTO com LTV)

Notebook de apoio didático para aula FP&A Banco BV.

Objetivos:
- carregar o dataset sintético de prepay
- validar taxa de evento e censura
- estimar curva de Kaplan-Meier global
- comparar curvas por faixas de LTV


In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.style.use('seaborn-v0_8-whitegrid')
pd.set_option('display.max_columns', 30)

In [ ]:
root = Path.cwd().resolve()
if root.name == 'notebooks':
    project_root = root.parent
else:
    project_root = root

data_path = project_root / 'data' / 'bv_auto_prepay_survival_sintetico.csv'
df = pd.read_csv(data_path, parse_dates=['orig_dt'])

print(f'Dataset: {data_path}')
print(f'Linhas: {len(df):,}')
df.head(3)

In [ ]:
event_rate = df['E_prepay'].mean()
censor_rate = (df['E_prepay'] == 0).mean()

print(f'Taxa de evento observado: {event_rate:.2%}')
print(f'Taxa de censura: {censor_rate:.2%}')
print(f'T_meses -> min/mediana/max: {df["T_meses"].min()} / {df["T_meses"].median():.1f} / {df["T_meses"].max()}')

df[['ltv', 'selic_at_orig', 'taxa_mensal_contrato', 'pti']].describe()

## Funções auxiliares para Kaplan-Meier (sem bibliotecas externas)

In [ ]:
def km_curve(t, e):
    """Calcula S(t) de Kaplan-Meier para tempos discretos (meses)."""
    arr = pd.DataFrame({'t': np.asarray(t, dtype=int), 'e': np.asarray(e, dtype=int)})
    arr = arr[arr['t'] >= 0].copy()

    event_table = (
        arr.groupby('t', as_index=False)
        .agg(events=('e', 'sum'), total=('e', 'size'))
        .sort_values('t')
    )

    n = len(arr)
    surv = 1.0
    times = [0]
    surv_values = [1.0]

    for _, row in event_table.iterrows():
        t_i = int(row['t'])
        d_i = int(row['events'])
        n_i = n

        if n_i > 0 and d_i > 0:
            surv *= (1.0 - d_i / n_i)

        times.append(t_i)
        surv_values.append(surv)

        n -= int(row['total'])

    return pd.DataFrame({'t': times, 'S_t': surv_values})


def median_survival(km_df):
    below = km_df[km_df['S_t'] <= 0.5]
    if below.empty:
        return np.nan
    return int(below['t'].iloc[0])

## Curva KM global

In [ ]:
km_all = km_curve(df['T_meses'], df['E_prepay'])
med_all = median_survival(km_all)

fig, ax = plt.subplots(figsize=(9, 5))
ax.step(km_all['t'], km_all['S_t'], where='post', linewidth=2.2, label='Carteira total')
ax.set_title('Kaplan-Meier - Sobrevivência sem prepay (global)')
ax.set_xlabel('Meses desde originação')
ax.set_ylabel('S(t): probabilidade de não quitar antecipadamente')
ax.set_ylim(0, 1.02)
ax.legend()
plt.show()

print(f'Mediana de tempo até prepay (global): {med_all} meses')

## Curvas KM por regime de LTV

In [ ]:
df_plot = df.copy()
df_plot['ltv_faixa'] = pd.cut(
    df_plot['ltv'],
    bins=[0.0, 0.75, 0.95, np.inf],
    labels=['LTV < 0.75', '0.75 <= LTV <= 0.95', 'LTV > 0.95'],
    include_lowest=True,
)

summary = (
    df_plot.groupby('ltv_faixa', observed=False)
    .agg(
        contratos=('id', 'count'),
        evento_rate=('E_prepay', 'mean'),
        T_mediana=('T_meses', 'median')
    )
    .reset_index()
)
summary

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

for faixa, g in df_plot.groupby('ltv_faixa', observed=False):
    km_g = km_curve(g['T_meses'], g['E_prepay'])
    ax.step(km_g['t'], km_g['S_t'], where='post', linewidth=2.0, label=f'{faixa} (n={len(g):,})')

ax.set_title('Kaplan-Meier por faixa de LTV')
ax.set_xlabel('Meses desde originação')
ax.set_ylabel('S(t): probabilidade de não quitar antecipadamente')
ax.set_ylim(0, 1.02)
ax.legend()
plt.show()

## Leitura executiva (didática)

- Curvas mais baixas indicam maior velocidade de prepay.
- A segmentação por LTV mostra regimes diferentes de comportamento de quitação.
- Esse diagnóstico pode apoiar projeção de duração da carteira e cenários de refinanciamento.